In [10]:
import datetime as dt
from utility import (
    read_sql,
    fd_basicinfo,
    fd_typeclass,
    fd_hshkiport,
    fd_assetportfolio,
    nearest_report_date
)
import pandas as pd

In [11]:
# 获取回溯的报告日

today = dt.datetime.today()
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [12]:
# 获取所有目标混合型基金（清算结束日期晚于当前日）

security_ids = read_sql(f"""
select SECURITYID from TQ_FD_TYPECLASS 
WHERE
    ISVALID = 1 AND
    L1CODE = 2 AND
    (ENDDATE>='{report_dates_begin}' or ENDDATE = '19000101') 
ORDER BY SECURITYID;
""")["SECURITYID"].unique().tolist()

In [13]:
# 获取相关组合情况

basic_info = fd_basicinfo(security_ids, current_date)
type_info = fd_typeclass(security_ids, current_date)
hkport_info = fd_hshkiport(security_ids, report_dates_begin)
hkport_info = hkport_info[hkport_info.INDUSTRYNAME == "合计"].groupby(["SECURITYID"], as_index=False)[["ACCNETMKTCAP"]].mean()
asset_port = fd_assetportfolio(security_ids, report_dates_begin).groupby("SECURITYID", as_index=False)[["EQUITYINVERTO"]].mean()

In [14]:
df = pd.merge(basic_info, type_info, on="SECURITYID")
df = pd.merge(df, asset_port, on="SECURITYID", how="left")
df = pd.merge(df, hkport_info, on="SECURITYID", how="left")
df["ACCNETMKTCAP"] = df["ACCNETMKTCAP"].fillna(0)

In [18]:
# 分类
df.loc[df["EQUITYINVERTO"] >= 70, "混合型（子类）"] = "高权益仓位混合型基金"
df.loc[((df["EQUITYINVERTO"] >= 50) & (df["EQUITYINVERTO"] < 70)), "混合型（子类）"] = "中高权益仓位混合型基金"
df.loc[((df["EQUITYINVERTO"] >= 30) & (df["EQUITYINVERTO"] < 50)), "混合型（子类）"] = "中权益仓位混合型基金"
df.loc[((df["EQUITYINVERTO"] >= 0) & (df["EQUITYINVERTO"] < 30)), "混合型（子类）"] = "低权益仓位混合型基金"
df.loc[(df["ACCNETMKTCAP"] / df["EQUITYINVERTO"] > 0.5), "混合型（子类）"] = "港股通混合型基金"

In [19]:
df.groupby("混合型（子类）")[["FSYMBOL"]].count()

,FSYMBOL
混合型（子类）,
中权益仓位混合型基金,167
中高权益仓位混合型基金,240
低权益仓位混合型基金,625
港股通混合型基金,17
高权益仓位混合型基金,1661
